In [ ]:
import numpy as np
import copy
import sys

In [ ]:
# Transforma a string em uma matriz, substituindo _ por 0
def string_to_puzzle(str_puzzle):
  l = [int(i) if i.isdigit() else 0 for i in str_puzzle]
  m_puzzle = np.array(l).reshape((3,3))

  return m_puzzle
  
# Transforma a matriz do puzzle em uma string
def puzzle_to_string(puzzle_matrix):
  puzzle_string = [str(x) if x > 0 else '_' for i in puzzle_matrix for x in i]
  puzzle_string = ''.join(puzzle_string)

  return puzzle_string

In [ ]:
def distancia_de_hamming(stra,strb):
  c = 0
  for a,b in zip(stra,strb):
    c += 1 if a != b else 0
  return c

def distancia_manhattan(pos1,pos2):
  return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])

In [ ]:
def h1(state):
  goal_state = "12345678_"
  return distancia_de_hamming(state,goal_state)

def h2(state):
  s = 0
  goal_state = "12345678_"
  goal_matrix = string_to_puzzle(goal_state)
  current_state = string_to_puzzle(state)

  for number in goal_state:
    number_ = int(number) if number != '_' else 0
    numberpos_current_state = [x[0] for x in np.where(current_state == number_)]
    numberpos_goal_state = [x[0] for x in np.where(goal_matrix == number_)]
    
    s += distancia_manhattan(numberpos_current_state,numberpos_goal_state)
  
  return s

In [ ]:
class PuzzleNode:
  def __init__(self, parent_node, state, action, cost, children_node_list = []):
    self.parent_node = parent_node
    self.state = state
    self.action = action
    self.cost = cost
    self.children_node_list = children_node_list

  def __str__(self):
    return "({0},{1},{2},{3})".format(self.action, self.state, self.cost, self.parent_node.state)


In [ ]:
def move(puzzle_state_matrix, direction):
  puzzle_copy = puzzle_state_matrix.copy()
  row_position, column_position = [x[0] for x in np.where(puzzle_copy == 0)]

  if direction == "acima":
     puzzle_copy[row_position, column_position], puzzle_copy[row_position - 1, column_position] = puzzle_copy[row_position - 1, column_position], puzzle_copy[row_position, column_position]
     return puzzle_copy

  if direction == "abaixo":
    puzzle_copy[row_position, column_position], puzzle_copy[row_position + 1, column_position] = puzzle_copy[row_position + 1, column_position], puzzle_copy[row_position, column_position]
    return puzzle_copy

  if direction == "esquerda":
    puzzle_copy[row_position, column_position - 1], puzzle_copy[row_position, column_position] = puzzle_copy[row_position, column_position], puzzle_copy[row_position, column_position - 1]
    return puzzle_copy

  if direction == "direita":
    puzzle_copy[row_position, column_position + 1], puzzle_copy[row_position, column_position] = puzzle_copy[row_position, column_position], puzzle_copy[row_position, column_position + 1]
    return puzzle_copy
    
  

In [ ]:
def sucessor(puzzle_node):
  puzzle_matrix = string_to_puzzle(puzzle_node)

  row_position, column_position = [x[0] for x in np.where(puzzle_matrix == 0)]
  available_positions = []

  # Pode ir para cima
  if row_position > 0:
    new_position = ("acima", puzzle_to_string(move(puzzle_matrix, "acima")))
    available_positions.append(new_position)

  # Pode ir para baixo
  if row_position < 2:
    new_position = ("abaixo", puzzle_to_string(move(puzzle_matrix, "abaixo")))
    available_positions.append(new_position)

  # Pode ir para esquerda
  if column_position > 0:
    new_position = ("esquerda", puzzle_to_string(move(puzzle_matrix, "esquerda")))
    available_positions.append(new_position)

  # Pode ir para direita
  if column_position < 2:
    new_position = ("direita", puzzle_to_string(move(puzzle_matrix, "direita")))
    available_positions.append(new_position)

  return available_positions

In [ ]:
def expande(puzzle_node):
  puzzle_children = []
  children_tuple_list = sucessor(puzzle_node.state)

  for action, state in children_tuple_list:
    children_node = PuzzleNode(puzzle_node, state, action, puzzle_node.cost + 1)
    puzzle_children.append(children_node)

  return puzzle_children

In [ ]:
def avalia_bfs(initial_node):
  queue = []
  visited = set()
  first_children_list = expande(initial_node)
  queue += first_children_list

  while queue:
    current_node = queue.pop(0)
    visited.add(current_node.state)

    if current_node.state == "12345678_":
      break

    for n in expande(current_node):
      if n.state not in visited:
        queue.append(n)

  node_path = []
  while current_node.parent_node:
    node_path.append(current_node.action)
    current_node = current_node.parent_node  

  node_path.reverse()

  return node_path

In [ ]:
def avalia_dfs(initial_node):
  queue = []
  visited = set()
  first_children_list = expande(initial_node)
  queue += first_children_list

  while queue:
    current_node = queue.pop()
    visited.add(current_node.state)

    if current_node.state == "12345678_":
      break

    for n in expande(current_node):
      if n.state not in visited:
        queue.append(n)

  node_path = []
  while current_node.parent_node:
    node_path.append(current_node.action)
    current_node = current_node.parent_node  

  print(len(node_path))
  node_path.reverse()
  return node_path

In [ ]:
def avalia_astar_h1(initial_node):
  queue = []
  visited = set()
  first_children_list = expande(initial_node)

  for n in first_children_list:
    queue.append((n, h1(n.state)))

  # Loop de busca
  while queue:
    min_index = queue.index(min(queue, key = lambda t : t[1])) # Posição do elemento na lista com menor f (custo + heurística)
    current_node, f = queue.pop(min_index)

    visited.add(current_node.state)

    if current_node.state == "12345678_":
      break

    for n in expande(current_node):
      if n.state not in visited:
        queue.append((n, h1(n.state)))

  node_path = []
  while current_node.parent_node:
    node_path.append(current_node.action)
    current_node = current_node.parent_node  

  node_path.reverse()
  print(len(node_path))
  
  return node_path

In [ ]:
def avalia_astar_h2(initial_node):
  queue = []
  visited = set()
  first_children_list = expande(initial_node)

  for n in first_children_list:
    queue.append((n, h2(n.state)))

  # Loop de busca
  while queue:
    min_index = queue.index(min(queue, key = lambda t : t[1])) # Posição do elemento na lista com menor f (custo + heurística)
    current_node, f = queue.pop(min_index)

    visited.add(current_node.state)

    if current_node.state == "12345678_":
      break
   
    for n in expande(current_node):
      if n.state not in visited:
        queue.append((n, h2(n.state)))

  node_path = []
  while current_node.parent_node:
    node_path.append(current_node.action)
    current_node = current_node.parent_node  

  node_path.reverse()
  print(len(node_path))
  
  return node_path

In [ ]:
function_input = "avalia_astar_h2" # Equivale a sys.argv[1]
state_input = "2_3541687" # Equivale a sys.argv[2]

puzzle_node = PuzzleNode(None, state_input, "", 0)

if function_input == "avalia_sucessor":
  for st_tuple in sucessor(state_input):
    print("({0},{1})".format(st_tuple[0], st_tuple[1]), end=" ") # Printa tupla estado-movimento

elif function_input == "avalia_expande":
  puzzle_node.cost = 0 # sys.argv[3]

  for n in expande(puzzle_node):
    print(n, end=" ")

elif function_input == "avalia_bfs":
  print(' '.join(avalia_bfs(puzzle_node)))

elif function_input == "avalia_dfs":
  print(' '.join(avalia_dfs(puzzle_node)))

elif function_input == "avalia_astar_h1":
  print(' '.join(avalia_astar_h1(puzzle_node)))

elif function_input == "avalia_astar_h2":
  print(' '.join(avalia_astar_h2(puzzle_node)))

45
esquerda abaixo direita direita abaixo esquerda esquerda acima direita abaixo direita acima esquerda acima esquerda abaixo abaixo direita acima acima direita abaixo esquerda abaixo direita acima acima esquerda abaixo abaixo direita acima esquerda abaixo esquerda acima direita direita abaixo esquerda acima esquerda abaixo direita direita


In [ ]:
# function_input = sys.argv[1] # Função a ser executada
# state_input = sys.argv[2] # Parâmetro para a função
# puzzle_node = PuzzleNode(None, state_input, "", 0)

# if function_input == "avalia_sucessor":
#   for st_tuple in sucessor(state_input):
#     print("({0},{1})".format(st_tuple[0], st_tuple[1]), end=" ") # Printa tupla estado-movimento

# elif function_input == "avalia_expande":
#   puzzle_node.cost = int(sys.argv[3])

#   for n in expande(puzzle_node):
#     print(n, end=" ")

# elif function_input == "avalia_bfs":
#   print(' '.join(avalia_bfs(puzzle_node)))

# elif function_input == "avalia_dfs":
#   print(' '.join(avalia_dfs(puzzle_node)))

# elif function_input == "avalia_astar_h1":
#   print(' '.join(avalia_astar_h1(puzzle_node)))

# elif function_input == "avalia_astar_h2":
#   print(' '.join(avalia_astar_h2(puzzle_node)))